In [4]:
import pandas as pd
import numpy as np

# Funktion zur Berechnung der Distanz (Haversine-Formel)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Erdradius in Kilometern
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

airlines = pd.read_csv('../data/airlines.csv')
airports = pd.read_csv('../data/airports.csv')
routes = pd.read_csv('../data/routes.csv')

# Spaltennamen bereinigen
airlines.columns = ['id', 'name', 'alias', 'iata', 'icao', 'callsign', 'country', 'active']
airports.columns = ['id', 'name', 'city', 'country', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'timezone', 'dst', 'tz_database_time_zone', 'type', 'source']
routes.columns = ['airline', 'airline_id', 'source_airport', 'source_airport_id', 'destination_airport', 'destination_airport_id', 'codeshare', 'stops', 'equipment']

# Zusammenführung von Routen mit Flughafendaten
routes = routes.merge(airports[['iata', 'latitude', 'longitude', 'name', 'city', 'country']], 
                      left_on='source_airport', right_on='iata', how='left').rename(
                      columns={'latitude': 'source_lat', 'longitude': 'source_lon',
                               'name': 'source_name', 'city': 'source_city', 'country': 'source_country'})

routes = routes.merge(airports[['iata', 'latitude', 'longitude', 'name', 'city', 'country']], 
                      left_on='destination_airport', right_on='iata', how='left').rename(
                      columns={'latitude': 'dest_lat', 'longitude': 'dest_lon',
                               'name': 'dest_name', 'city': 'dest_city', 'country': 'dest_country'})

# Zusammenführung mit Airline-Daten
routes = routes.merge(airlines, left_on='airline', right_on='iata', how='left').rename(
                      columns={'name': 'airline_name', 'alias': 'airline_alias', 'icao': 'airline_icao',
                               'callsign': 'airline_callsign', 'country': 'airline_country', 'active': 'airline_active'})

# Berechnung der Distanz
routes['distance_km'] = haversine(routes['source_lat'], routes['source_lon'], routes['dest_lat'], routes['dest_lon'])

# Auswahl der relevanten Spalten
summary = routes[['source_airport', 'source_name', 'source_city', 'source_country', 'source_lat', 'source_lon',
                  'destination_airport', 'dest_name', 'dest_city', 'dest_country', 'dest_lat', 'dest_lon',
                  'airline_name', 'airline_alias', 'airline_icao', 'airline_callsign', 'airline_country', 'airline_active', 'stops', 'distance_km']]

# Ergebnis als CSV-Datei speichern
summary.to_csv('../data/flughafen_netzwerk.csv', index=False)

print("Daten erfolgreich zusammengefasst und gespeichert als 'flughafen_netzwerk.csv'")


Daten erfolgreich zusammengefasst und gespeichert als 'flughafen_netzwerk.csv'
